# Sparkify Project

In [0]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, countDistinct
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA,RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import re


# Load and Clean Dataset
 Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids.

In [0]:
#Load Dataset

# File location and type
file_location = "/FileStore/tables/medium_sparkify_event_data.json"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
Martin Orford,Logged In,Joseph,M,20,Morales,597.55057,free,"Corpus Christi, TX",PUT,NextSong,1532063507000,292,Grand Designs,200,1538352011000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",293
John Brown's Body,Logged In,Sawyer,M,74,Larson,380.21179,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1538069638000,97,Bulls,200,1538352025000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",98
Afroman,Logged In,Maverick,M,184,Santiago,202.37016,paid,"Orlando-Kissimmee-Sanford, FL",PUT,NextSong,1535953455000,178,Because I Got High,200,1538352118000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179
null,Logged In,Maverick,M,185,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",PUT,Logout,1535953455000,178,null,307,1538352119000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179
Lily Allen,Logged In,Gianna,F,22,Campos,194.53342,paid,"Mobile, AL",PUT,NextSong,1535931018000,245,Smile (Radio Edit),200,1538352124000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,246
Carter USM,Logged In,Sofia,F,266,Gordon,138.29179,paid,"Rochester, MN",PUT,NextSong,1533175710000,162,Airplane Food,200,1538352125000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",163
null,Logged Out,null,null,186,null,null,paid,null,GET,Home,null,178,null,200,1538352148000,null,
null,Logged Out,null,null,187,null,null,paid,null,GET,Home,null,178,null,200,1538352151000,null,
null,Logged Out,null,null,188,null,null,paid,null,GET,Home,null,178,null,200,1538352168000,null,
null,Logged Out,null,null,189,null,null,paid,null,PUT,Login,null,178,null,307,1538352169000,null,


In [0]:
# Shape of dataframe
print("Shape of dataframe : ",(df.count(), len(df.columns)))

Shape of dataframe : (543705, 18)

In [0]:
# Schema of dataframe
df.printSchema()

root
-- artist: string (nullable = true)
-- auth: string (nullable = true)
-- firstName: string (nullable = true)
-- gender: string (nullable = true)
-- itemInSession: long (nullable = true)
-- lastName: string (nullable = true)
-- length: double (nullable = true)
-- level: string (nullable = true)
-- location: string (nullable = true)
-- method: string (nullable = true)
-- page: string (nullable = true)
-- registration: long (nullable = true)
-- sessionId: long (nullable = true)
-- song: string (nullable = true)
-- status: long (nullable = true)
-- ts: long (nullable = true)
-- userAgent: string (nullable = true)
-- userId: string (nullable = true)

In [0]:
# Summary Statistics
display(df.describe())

summary,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
count,432877,543705,528005,528005,543705,528005,432877,543705,528005,543705,543705,528005,543705,432877,543705,543705,528005,543705
mean,527.5289537712895,null,null,null,107.30629109535502,null,248.66459278007346,null,null,null,null,1.5355234148639202E12,2040.8143533717732,NaN,210.01829116892432,1.540964541209823E12,null,60268.42669103512
stddev,966.1072451772757,null,null,null,116.72350849188157,null,98.41266955052019,null,null,null,null,3.078725492995963E9,1434.3389310782693,NaN,31.471919021567253,1.482057144910989E9,null,109898.82324176563
min,!!!,Cancelled,Aaliyah,F,0,Abbott,0.78322,free,"Akron, OH",GET,About,1509854193000,1,ÃÂg ÃÂtti GrÃÂ¡a ÃÂsku,200,1538352011000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.3 (KHTML, like Gecko) Version/8.0 Safari/600.1.3""",
max,ÃÂlafur Arnalds,Logged Out,Zyonna,M,1005,Wright,3024.66567,paid,"York-Hanover, PA",PUT,Upgrade,1543073874000,4808,ÃÂ¾etta Gerist ÃÂ¡ Bestu BÃÂ¦jum,404,1543622466000,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0),99


Observations/Abnormalities in data : 1)Many columns have Null values 2)Song has Nan values 3)Many columns has invalid data values 4)UserId has some empty string values

In [0]:
# Exploration of rows where userId is empty string

# Count of rows with userId as empty string
display(df.filter('userId = ""').groupby().count())

# Looking at users with rows with userId as empty string
display(df.filter('userId = ""'))

# Calculate distict UserID for 'Logged Out' auth users 
display(df.filter('auth = "Logged Out"').select(countDistinct("userID")))


count
15700


artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
null,Logged Out,null,null,186,null,null,paid,null,GET,Home,null,178,null,200,1538352148000,null,
null,Logged Out,null,null,187,null,null,paid,null,GET,Home,null,178,null,200,1538352151000,null,
null,Logged Out,null,null,188,null,null,paid,null,GET,Home,null,178,null,200,1538352168000,null,
null,Logged Out,null,null,189,null,null,paid,null,PUT,Login,null,178,null,307,1538352169000,null,
null,Logged Out,null,null,114,null,null,free,null,GET,Home,null,442,null,200,1538353292000,null,
null,Logged Out,null,null,34,null,null,free,null,GET,Home,null,292,null,200,1538355024000,null,
null,Logged Out,null,null,35,null,null,free,null,GET,Help,null,292,null,200,1538355098000,null,
null,Logged Out,null,null,36,null,null,free,null,GET,Home,null,292,null,200,1538355178000,null,
null,Logged Out,null,null,37,null,null,free,null,GET,About,null,292,null,200,1538355186000,null,
null,Logged Out,null,null,38,null,null,free,null,PUT,Login,null,292,null,307,1538355187000,null,


count(DISTINCT userID)
1


Observation: For Logged Out users, userId is empty string

In [0]:
#Checking duplicate rows
df2 = df.dropDuplicates()
print("Distinct count: ",df2.count())

# Total number of rows in Dataframe
print("Dataframe row count: ",df.count())

Distinct count: 543705
Dataframe row count: 543705

In [0]:
# Get count of nan values
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+

In [0]:
# Get count of null values
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+
artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+
110828| 0| 15700| 15700| 0| 15700|110828| 0| 15700| 0| 0| 15700| 0|110828| 0| 0| 15700| 0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+------+------+---+---------+------+

In [0]:
## Get count of null values where user are logged out.
df.where('auth == "Logged Out" ').select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
 15606| 0| 15606| 15606| 0| 15606| 15606| 0| 15606| 0| 0| 15606| 0|15606| 0| 0| 15606| 0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+

Observation : Most of the Null values are from users which are logged out.

In [0]:
# Number of distinct page values
df.select(countDistinct("page")).show()

# Number Of records per page type
display(df.groupby("page").count())


+--------------------+
count(DISTINCT page)|
+--------------------+
 22|
+--------------------+

page,count
Submit Downgrade,117
Thumbs Down,4911
Home,27412
Downgrade,3811
Roll Advert,7773
Logout,5990
Save Settings,585
About,1855
Settings,2964
Login,6011


In [0]:
# Bar Plot for number Of records per page type
display(df.groupby("page").count())

page,count
Cancel,99
Submit Downgrade,117
Thumbs Down,4911
Home,27412
Downgrade,3811
Roll Advert,7773
Logout,5990
Save Settings,585
Cancellation Confirmation,99
About,1855


In [0]:
# Number Of records per auth type
display(df.groupby("auth").count())

auth,count
Logged Out,15606
Cancelled,99
Logged In,527906
Guest,94


In [0]:
# Bar plot for number Of records per auth type
display(df.groupby("auth").count())

auth,count
Logged Out,15606
Logged In,527906
Cancelled,99
Guest,94


In [0]:
# Number of unique songs
df.select(countDistinct("song")).show()

+--------------------+
count(DISTINCT song)|
+--------------------+
 80292|
+--------------------+

In [0]:
# Number of unique artists
df.select(countDistinct("artist")).show()

+----------------------+
count(DISTINCT artist)|
+----------------------+
 21247|
+----------------------+

In [0]:
# Number Of records per level type
display(df.groupby("level").count())

level,count
free,115108
paid,428597


In [0]:
# Number Of records per method type
display(df.groupby("method").count())

method,count
PUT,495143
GET,48562


In [0]:
# Convert ts to datetime format
import datetime
get_time= udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%m-%d-%Y %H:%M:%S"))
data = df.withColumn("ts", get_time(df.ts))
display(data.select("ts"))

ts
10-01-2018 00:00:11
10-01-2018 00:00:25
10-01-2018 00:01:58
10-01-2018 00:01:59
10-01-2018 00:02:04
10-01-2018 00:02:05
10-01-2018 00:02:28
10-01-2018 00:02:31
10-01-2018 00:02:48
10-01-2018 00:02:49


In [0]:
# Count of "Cancel","Cancellation Confirmation","Downgrade","Submit Downgrade" pages
data.filter(col("page").isin(["Cancel","Cancellation Confirmation","Downgrade","Submit Downgrade"])).groupby("page").count().show()

+--------------------+-----+
 page|count|
+--------------------+-----+
 Cancel| 99|
 Submit Downgrade| 117|
 Downgrade| 3811|
Cancellation Conf...| 99|
+--------------------+-----+

In [0]:
# Filter certain pages data which might be considered as Churn
display(data.filter(col("page").isin(["Cancel","Cancellation Confirmation","Downgrade", "Submit Downgrade"]))
                                .select("auth","gender","itemInSession","location","level","page","sessionid","status","ts","userId"))

auth,gender,itemInSession,location,level,page,sessionid,status,ts,userId
Logged In,F,268,"Rochester, MN",paid,Downgrade,162,200,10-01-2018 00:05:36,163
Logged In,F,296,"Rochester, MN",paid,Downgrade,162,200,10-01-2018 01:44:18,163
Logged In,F,85,"Mobile, AL",paid,Downgrade,245,200,10-01-2018 03:11:50,246
Logged In,F,15,"Dallas-Fort Worth-Arlington, TX",paid,Downgrade,418,200,10-01-2018 03:58:02,127
Logged In,F,131,"Mobile, AL",paid,Downgrade,245,200,10-01-2018 05:47:16,246
Logged In,F,53,"Dallas-Fort Worth-Arlington, TX",paid,Downgrade,418,200,10-01-2018 06:23:51,127
Logged In,M,34,"Kansas City, MO-KS",paid,Downgrade,476,200,10-01-2018 06:42:35,18
Logged In,M,91,"Chicago-Naperville-Elgin, IL-IN-WI",paid,Downgrade,287,200,10-01-2018 07:16:30,288
Logged In,M,16,"Lexington-Fayette, KY",paid,Downgrade,493,200,10-01-2018 07:24:39,51
Logged In,M,105,"Chicago-Naperville-Elgin, IL-IN-WI",paid,Downgrade,287,200,10-01-2018 07:50:45,288


Observation : After "cancellation" event page there is always a "cancellation confirmation" event page

In [0]:
# filtering data for one user which "submit downgrade"
display(data.select("auth","gender","itemInSession","location","level","page","sessionid","status","ts","userId").where("userId ==127"))

auth,gender,itemInSession,location,level,page,sessionid,status,ts,userId
Logged In,F,0,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:03:08,127
Logged In,F,1,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:06:50,127
Logged In,F,2,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:10:29,127
Logged In,F,3,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:15:28,127
Logged In,F,4,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:18:48,127
Logged In,F,5,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:22:43,127
Logged In,F,6,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:26:18,127
Logged In,F,7,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:30:11,127
Logged In,F,8,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:33:59,127
Logged In,F,9,"Dallas-Fort Worth-Arlington, TX",paid,NextSong,418,200,10-01-2018 03:36:06,127


Observation : After "downgrade" event, there is no changed observed for user but after "submit downgrade" event, level changes from paid to free.

# Exploratory Data Analysis

### Define Churn
Based on preliminary analysis above, we will be consider `Cancellation Confirmation` and `SubmitDowngrade` events as `Churn` event.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [0]:
# Adding column for highlighting churn events in the dataframe.
Is_Cancellation_Confirmation = udf(lambda x : 1 if x=="Cancellation Confirmation" else 0)
Is_SubmitDowngrade = udf(lambda x : 1 if x=="Submit Downgrade" else 0)
Is_Churn = udf(lambda x : 1 if (x=="Submit Downgrade" or x=="Cancellation Confirmation")  else 0)

data = data.withColumn("Is_Cancellation_Confirmation", Is_Cancellation_Confirmation(data.page))
data = data.withColumn("Is_SubmitDowngrade", Is_SubmitDowngrade(data.page))
data = data.withColumn("Is_Churn", Is_Churn(data.page))
display(data)

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,Is_Cancellation_Confirmation,Is_SubmitDowngrade,Is_Churn
Martin Orford,Logged In,Joseph,M,20,Morales,597.55057,free,"Corpus Christi, TX",PUT,NextSong,1532063507000,292,Grand Designs,200,10-01-2018 00:00:11,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",293,0,0,0
John Brown's Body,Logged In,Sawyer,M,74,Larson,380.21179,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1538069638000,97,Bulls,200,10-01-2018 00:00:25,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",98,0,0,0
Afroman,Logged In,Maverick,M,184,Santiago,202.37016,paid,"Orlando-Kissimmee-Sanford, FL",PUT,NextSong,1535953455000,178,Because I Got High,200,10-01-2018 00:01:58,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
null,Logged In,Maverick,M,185,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",PUT,Logout,1535953455000,178,null,307,10-01-2018 00:01:59,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
Lily Allen,Logged In,Gianna,F,22,Campos,194.53342,paid,"Mobile, AL",PUT,NextSong,1535931018000,245,Smile (Radio Edit),200,10-01-2018 00:02:04,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,246,0,0,0
Carter USM,Logged In,Sofia,F,266,Gordon,138.29179,paid,"Rochester, MN",PUT,NextSong,1533175710000,162,Airplane Food,200,10-01-2018 00:02:05,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",163,0,0,0
null,Logged Out,null,null,186,null,null,paid,null,GET,Home,null,178,null,200,10-01-2018 00:02:28,null,,0,0,0
null,Logged Out,null,null,187,null,null,paid,null,GET,Home,null,178,null,200,10-01-2018 00:02:31,null,,0,0,0
null,Logged Out,null,null,188,null,null,paid,null,GET,Home,null,178,null,200,10-01-2018 00:02:48,null,,0,0,0
null,Logged Out,null,null,189,null,null,paid,null,PUT,Login,null,178,null,307,10-01-2018 00:02:49,null,,0,0,0


In [0]:
# Look at Churn events data
display(data.where("Is_Churn == 1"))

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,Is_Cancellation_Confirmation,Is_SubmitDowngrade,Is_Churn
null,Logged In,Carter,M,117,Cook,null,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,Submit Downgrade,1522793334000,287,null,307,10-01-2018 08:24:07,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53""",288,0,1,1
null,Cancelled,Olivia,F,40,Carr,null,free,"Fort Wayne, IN",GET,Cancellation Confirmation,1536758439000,490,null,200,10-01-2018 13:30:16,Mozilla/5.0 (Windows NT 6.2; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,208,1,0,1
null,Logged In,Aurora,F,105,Humphrey,null,paid,"Dallas-Fort Worth-Arlington, TX",PUT,Submit Downgrade,1536795126000,537,null,307,10-02-2018 02:38:35,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",127,0,1,1
null,Cancelled,Lillian,F,234,Cameron,null,paid,"Columbus, OH",GET,Cancellation Confirmation,1533472700000,471,null,200,10-02-2018 12:19:53,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,231,1,0,1
null,Cancelled,Alex,M,109,Myers,null,paid,"Grand Rapids-Wyoming, MI",GET,Cancellation Confirmation,1529995579000,682,null,200,10-03-2018 04:11:11,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",236,1,0,1
null,Logged In,Adam,M,10,Johnson,null,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,Submit Downgrade,1536986118000,723,null,307,10-03-2018 08:16:06,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",174,0,1,1
null,Cancelled,Rafael,M,66,Crawford,null,free,"Bowling Green, KY",GET,Cancellation Confirmation,1533887620000,628,null,200,10-03-2018 18:48:49,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",156,1,0,1
null,Logged In,Bryson,M,127,Roberson,null,paid,"Houston-The Woodlands-Sugar Land, TX",PUT,Submit Downgrade,1521380675000,826,null,307,10-05-2018 12:26:57,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",6,0,1,1
null,Logged In,Joseph,M,125,Morales,null,paid,"Corpus Christi, TX",PUT,Submit Downgrade,1532063507000,899,null,307,10-05-2018 14:05:23,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",293,0,1,1
null,Logged In,Lorelei,F,142,Silva,null,paid,"Pittsburgh, PA",PUT,Submit Downgrade,1534779204000,393,null,307,10-05-2018 19:34:36,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",145,0,1,1


In [0]:
# Number of Churn vs Non Churn events
display(data.groupBy("Is_Churn").count())

Is_Churn,count
0,543489
1,216


In [0]:
# Display of Number of Churn vs Non Churn events
display(data.groupBy("Is_Churn").count())

Is_Churn,count
0,543489
1,216


In [0]:
# Distribution of Churn events among "Cancellation Confirmation" and "Submit Downgrade"
display(data.filter(col("page").isin(["Cancellation Confirmation","Submit Downgrade"])).groupby("page").count())

page,count
Submit Downgrade,117
Cancellation Confirmation,99


In [0]:
# Distribution of Is_Churn amoung auth catergories
display(data.groupby("Is_Churn","Is_Cancellation_Confirmation","Is_SubmitDowngrade").pivot("auth").count())

Is_Churn,Is_Cancellation_Confirmation,Is_SubmitDowngrade,Cancelled,Guest,Logged In,Logged Out
0,0,0,null,94,527789,15606
1,0,1,null,null,117,null
1,1,0,99,null,null,null


Observation:

In [0]:
# bar plt for distribution of Is_Churn among auth catergories
display(data.groupby("Is_Churn").pivot("auth").count())

Is_Churn,Cancelled,Guest,Logged In,Logged Out
0,null,94,527789,15606
1,99,null,117,null


Observation : 1) All the Logged Out users are non Churn events, therefore these are not providing any beneficial information, therefore rows can be removed from the data.  
2) During Cancellation event, the auth is always cancelled and during downgrade event the auth is always Logged in

In [0]:
# Removing Logged out data from data frame
data_not_logout = data.where("auth != 'Logged Out' ")
display(data_not_logout)

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,Is_Cancellation_Confirmation,Is_SubmitDowngrade,Is_Churn
Martin Orford,Logged In,Joseph,M,20,Morales,597.55057,free,"Corpus Christi, TX",PUT,NextSong,1532063507000,292,Grand Designs,200,10-01-2018 00:00:11,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",293,0,0,0
John Brown's Body,Logged In,Sawyer,M,74,Larson,380.21179,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1538069638000,97,Bulls,200,10-01-2018 00:00:25,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",98,0,0,0
Afroman,Logged In,Maverick,M,184,Santiago,202.37016,paid,"Orlando-Kissimmee-Sanford, FL",PUT,NextSong,1535953455000,178,Because I Got High,200,10-01-2018 00:01:58,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
null,Logged In,Maverick,M,185,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",PUT,Logout,1535953455000,178,null,307,10-01-2018 00:01:59,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
Lily Allen,Logged In,Gianna,F,22,Campos,194.53342,paid,"Mobile, AL",PUT,NextSong,1535931018000,245,Smile (Radio Edit),200,10-01-2018 00:02:04,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,246,0,0,0
Carter USM,Logged In,Sofia,F,266,Gordon,138.29179,paid,"Rochester, MN",PUT,NextSong,1533175710000,162,Airplane Food,200,10-01-2018 00:02:05,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",163,0,0,0
null,Logged In,Maverick,M,190,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",GET,Home,1535953455000,178,null,200,10-01-2018 00:02:56,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
Aerosmith,Logged In,Lacey,F,107,Castaneda,220.39465,free,"El Campo, TX",PUT,NextSong,1537536032000,442,Walk This Way,200,10-01-2018 00:03:35,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",175,0,0,0
null,Logged In,Colin,M,0,Larson,null,free,"Dallas-Fort Worth-Arlington, TX",GET,Home,1537982255000,497,null,200,10-01-2018 00:04:01,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0,100,0,0,0
Amy Winehouse,Logged In,Colin,M,1,Larson,201.50812,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1537982255000,497,Teach Me Tonight,200,10-01-2018 00:04:19,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0,100,0,0,0


In [0]:
# Distribution of Is_Churn amoung gender catergories
display(data_not_logout.groupby("Is_Churn","Is_Cancellation_Confirmation","Is_SubmitDowngrade").pivot("gender").count())

Is_Churn,Is_Cancellation_Confirmation,Is_SubmitDowngrade,null,F,M
0,0,0,94,225292,302497
1,0,1,null,56,61
1,1,0,null,45,54


In [0]:
# Distribution of Is_Churn among gender catergories
display(data_not_logout.groupby("Is_Churn").pivot("gender").count())

Is_Churn,null,F,M
0,94,225292,302497
1,null,101,115


Observation : Distribition of gender is almost same among Churn and Non Churn events

In [0]:
# Distribution of Is_Churn amoung level catergories
display(data_not_logout.groupby("Is_Churn","Is_Cancellation_Confirmation","Is_SubmitDowngrade").pivot("level").count())

Is_Churn,Is_Cancellation_Confirmation,Is_SubmitDowngrade,free,paid
0,0,0,110023,417860
1,0,1,null,117
1,1,0,32,67


Observation :Downgrade Churn is happening with paid users and for Cancellation Churn, free users are two time more likely to churn

In [0]:
# Distribution of Is_Churn amoung level catergories
display(data_not_logout.groupby("Is_Churn").pivot("level").count())

Is_Churn,free,paid
0,110023,417860
1,32,184


In [0]:
display(data_not_logout)

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,Is_Cancellation_Confirmation,Is_SubmitDowngrade,Is_Churn
Martin Orford,Logged In,Joseph,M,20,Morales,597.55057,free,"Corpus Christi, TX",PUT,NextSong,1532063507000,292,Grand Designs,200,10-01-2018 00:00:11,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",293,0,0,0
John Brown's Body,Logged In,Sawyer,M,74,Larson,380.21179,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1538069638000,97,Bulls,200,10-01-2018 00:00:25,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",98,0,0,0
Afroman,Logged In,Maverick,M,184,Santiago,202.37016,paid,"Orlando-Kissimmee-Sanford, FL",PUT,NextSong,1535953455000,178,Because I Got High,200,10-01-2018 00:01:58,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
null,Logged In,Maverick,M,185,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",PUT,Logout,1535953455000,178,null,307,10-01-2018 00:01:59,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
Lily Allen,Logged In,Gianna,F,22,Campos,194.53342,paid,"Mobile, AL",PUT,NextSong,1535931018000,245,Smile (Radio Edit),200,10-01-2018 00:02:04,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0,246,0,0,0
Carter USM,Logged In,Sofia,F,266,Gordon,138.29179,paid,"Rochester, MN",PUT,NextSong,1533175710000,162,Airplane Food,200,10-01-2018 00:02:05,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",163,0,0,0
null,Logged In,Maverick,M,190,Santiago,null,paid,"Orlando-Kissimmee-Sanford, FL",GET,Home,1535953455000,178,null,200,10-01-2018 00:02:56,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",179,0,0,0
Aerosmith,Logged In,Lacey,F,107,Castaneda,220.39465,free,"El Campo, TX",PUT,NextSong,1537536032000,442,Walk This Way,200,10-01-2018 00:03:35,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36""",175,0,0,0
null,Logged In,Colin,M,0,Larson,null,free,"Dallas-Fort Worth-Arlington, TX",GET,Home,1537982255000,497,null,200,10-01-2018 00:04:01,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0,100,0,0,0
Amy Winehouse,Logged In,Colin,M,1,Larson,201.50812,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1537982255000,497,Teach Me Tonight,200,10-01-2018 00:04:19,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0,100,0,0,0


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.